In [1]:
import sys
sys.path.append("../../")
from radarange_orchestrator import llm

In [2]:
from radarange_orchestrator.tools import pdf_tool, ls_tool
m = llm()
chat = m.chat(tools = [pdf_tool, ls_tool])

Connecting to host: 95.165.10.219:1234


In [3]:
path = "downloads/Drofa_NV_centers_2023.pdf"
prompt = """
В папке downloads лежит статья в формате pdf. Заполни информацию о ссылках из неё.
"""
chat.add_user_message(prompt)
chat.display_thoughts()

In [5]:
from pydantic import BaseModel, RootModel
import json

from radarange_orchestrator.formatting import ResponseFormat

class Article(BaseModel):
    """Reference information, including authors, title, journal and date"""
    authors: str
    title: str
    journal: str
    issue_number_page: str | None = None
    year: int | None = None

class ArticleList(RootModel[list[Article]]):
    """References list of a science article"""
    root: list[Article]


response_format = ResponseFormat(json_schema=ArticleList)
print(response_format)

{
  "description": "References list of a science article",
  "items": {
    "description": "Reference information, including authors, title, journal and date",
    "properties": {
      "authors": {
        "title": "Authors",
        "type": "string"
      },
      "title": {
        "title": "Title",
        "type": "string"
      },
      "journal": {
        "title": "Journal",
        "type": "string"
      },
      "issue_number_page": {
        "anyOf": [
          {
            "type": "string"
          },
          {
            "type": "null"
          }
        ],
        "default": null,
        "title": "Issue Number Page"
      },
      "year": {
        "anyOf": [
          {
            "type": "integer"
          },
          {
            "type": "null"
          }
        ],
        "default": null,
        "title": "Year"
      }
    },
    "required": [
      "authors",
      "title",
      "journal"
    ],
    "title": "Article",
    "type": "object"
  },
  "titl

In [18]:
from pydantic import ValidationError

def validate_i(r: str) -> bool:
    try:
        Article.model_validate_json(r)
    except ValidationError:
        return False
    return True

def validate(r: str) -> bool:
    try:
        ArticleList.model_validate_json(r)
    except ValidationError:
        return False
    return True

In [7]:
ans = m.act(
    prompt=chat,
    max_tokens_per_message=10000,
    temperature=0,
    on_message=chat.append,
    max_prediction_rounds=5,
    response_format=response_format,
)
chat.display_thoughts()

Called list_directory with path: downloads
Called pdf_read with path: downloads/Drofa_NV_centers_2023.pdf
Taken 0.3 seconds to complete.


In [45]:
from radarange_orchestrator.utils.extract_tool_calls import remove_think_block


text = remove_think_block(ans.content)
print(text)




[
  {
    "authors": "J. M. Taylor, P. Cappellaro, L. Childress, L. Jiang, D. Budker, P. R. Hemmer, A. Yacoby, R. Walsworth, and M. D. Lukin",
    "title": "High-sensitivity diamond magnetometer with nanoscale resolution",
    "journal": "Nat Phys",
    "issue_number_page": "4, 810",
    "year": 2008
  },
  {
    "authors": "J. R. Maze et al.",
    "title": "Nanoscale magnetic sensing with an individual electronic spin in diamond",
    "journal": "Nature",
    "issue_number_page": "455, 644",
    "year": 2008
  },
  {
    "authors": "A. Jenkins, M. Pelliccione, G. Yu, X. Ma, X. Li, K. L. Wang, and A. C. B. Jayich",
    "title": "Single-spin sensing of domain-wall structure and dynamics in a thin-film skyrmion host",
    "journal": "Phys Rev Mater",
    "issue_number_page": "3, 083801",
    "year": 2019
  },
  {
    "authors": "M. L. Palm et al.",
    "title": "Imaging of Submicroampere Currents in Bilayer Graphene Using a Scanning Diamond Magnetometer",
    "journal": "Phys Rev Appl

In [46]:
validate(text)

False

In [66]:
from radarange_orchestrator.formatting import JsonType

def empty_to_none(J: JsonType, allowed_keys: dict[str] = []) -> JsonType:
    if isinstance(J, dict):
        for key in J.keys():
            if key in allowed_keys or allowed_keys == []:
                J[key] = empty_to_none(J[key], allowed_keys)
    elif isinstance(J, list):
        return [empty_to_none(I1, allowed_keys) for I1 in J]
    elif isinstance(J, str):
        return None if J == "" else J
    return J

def none_to_empty(J: JsonType, allowed_keys: dict[str] = []) -> JsonType:
    if isinstance(J, dict):
        for key in J.keys():
            if key in allowed_keys or allowed_keys == []:
                J[key] = none_to_empty(J[key], allowed_keys)
    elif isinstance(J, list):
        return [none_to_empty(I1, allowed_keys) for I1 in J]
    elif J is None:
        return ""
    return J

In [67]:
jj = json.loads('''{
  "authors": "E. Arimondo",
  "title": "Relaxation processes in coherent-population trapping",
  "journal": null,
  "issue_number_page": "2216",
  "year": 1996
}''')

print(jj)
jjj = empty_to_none(jj, 'year')
jjj = none_to_empty(jjj, 'journal')
print(jjj)

{'authors': 'E. Arimondo', 'title': 'Relaxation processes in coherent-population trapping', 'journal': None, 'issue_number_page': '2216', 'year': 1996}
{'authors': 'E. Arimondo', 'title': 'Relaxation processes in coherent-population trapping', 'journal': '', 'issue_number_page': '2216', 'year': 1996}


In [74]:
jans = json.loads(text)
jparsed = empty_to_none(json.loads(text), 'year')
jparsed = none_to_empty(jparsed, 'journal')

In [75]:
for j in jans:
    if not validate_i(json.dumps(j)):
        print(json.dumps(j, indent=2))

{
  "authors": "J. Kitching, L. Hollberg, S. Knappe, and R. Wynands",
  "title": "Compact Atomic Clock Based on Coherent Population Trapping",
  "journal": "Electronics Letters.",
  "issue_number_page": "",
  "year": ""
}


In [76]:
for j in jparsed:
    if not validate_i(json.dumps(j)):
        print(json.dumps(j, indent=2))

In [77]:
from radarange_orchestrator.utils.display import display_message


display_message(chat[-1])

In [78]:
chat.display_thoughts()